In [ ]:
import pandas as pd

from ML.utils.stat_tests import get_merged_texts
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from Text import Text
from ML.utils.stat_tests import get_authors_texts

In [64]:
def predict_author(p_values, authors):
    p_values = pd.Series(p_values)
    index_max_pv = p_values.argmax()
    author = authors[index_max_pv]
    return author

In [65]:
def khmelev_statistics(text1, text2):   #text1 == get_merged(df), text2 == text_for_attribution
    p_values_khmelev = []
    for text in text1["texts"]:
        p_values_khmelev.append((-1) * Text.one_sided_khmelev_statistics(Text(text), Text(text2)))
    return p_values_khmelev

def invariant_attribution_test(text1, text2):    #text1 == get_merged(df), text2 == text_for_attribution
    p_values_inv_attr_test = []
    for text in text1["texts"]:
        p_values_inv_attr_test.append(Text.invariant_attribution_test(Text(text), Text(text2))[1])
    return p_values_inv_attr_test

def burrows_delta(text1, text2):    #text1 == get_merged(df), text2 == text_for_attribution
    p_values_burrow_delta = []
    for text in text1["texts"]:
        p_values_burrow_delta.append((-1) * Text.burrows_delta(Text(text), Text(text2)))
    return p_values_burrow_delta

def unique_word_process_test(text1, text2):
    values = []
    for text in text1["texts"]:
        values.append(Text.unique_word_process_test_2_texts(Text(text), Text(text2))[1])

    return values

In [66]:
def cross_val_stats(df, statmethod):
    accuracy = []
    for shift in tqdm(range(0, 10, 2)):
        predicted_authors = []
        authors_seen = []
        test_ixs = []
        for ixs in range(len(df)):
            aut = df['targets'][ixs]
            if authors_seen.count(aut) < 2:
                authors_seen += [aut]
                test_ixs += [ixs + shift]
        train_ixs = list(set(range(len(df))) - set(test_ixs))
        authors_texts_test = df.iloc[test_ixs]
        authors_texts_train = get_merged_texts(df.iloc[train_ixs])
        #стат методы
        for book in tqdm(authors_texts_test['texts']):
            p_values = statmethod(authors_texts_train, book)
            predicted_authors.append(predict_author(p_values, authors_texts_train['targets']))
        accuracy.append(accuracy_score(predicted_authors, authors_texts_test['targets']))

    return accuracy


In [ ]:
df = get_authors_texts('C:\\Users\\mashk\\PycharmProjects\\texts_attribution\\Данные')

In [ ]:
cross_val_stats(df, khmelev_statistics)

  0%|          | 0/100 [00:00<?, ?it/s]